In [ ]:
# merge video 3 -> 1 && display the result video
from moviepy.editor import VideoFileClip
from moviepy.editor import *
from PIL import Image
import numpy as  np
from PIL import Image, ImageDraw,ImageFont
import ipywidgets as widgets
import glob, os, sys
from IPython.display import HTML
from base64 import b64encode
import cv2 

def add_text(image,text):
    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image)
    draw.text((5,5), text, fill=(255, 0, 0),font=font)
    return np.array(image)


def find_latest_video(directory):

    # Supported video file extensions
    video_extensions = {'.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv'}

    latest_video = None
    latest_time = 0

    # Walk through all files and directories within the given directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is a video
            if file.lower().endswith(tuple(video_extensions)):
                file_path = os.path.join(root, file)
                file_time = os.path.getmtime(file_path)

                # Update the latest video file
                if file_time > latest_time:
                    latest_video = file_path
                    latest_time = file_time

    return latest_video


latest_video = find_latest_video(directory='')

# video path
video1_list = ['']
video2_list = ['']
video3_list = [latest_video]

video_list = [video1_list, video2_list, video3_list]
print(f"video_list:{video_list}")

# ImageFont should download  
font = ImageFont.truetype("Montserrat-Black.otf", size=45)    
text_map = {'0':'title 1','1':'title 2','2':'title 3'}
    
for i in range(len(video_list[0])):
    all_frames = []
    for j in range(len(video_list)):
        
        video_path = video_list[j][i]
        clip = VideoFileClip(video_path)
        clip.fps = 30

        frames = [frame for frame in clip.iter_frames()]
        frames = [add_text(f,text_map[str(j)]) for f in frames ]
        all_frames.append(frames)

    new_all_frames = []
    min_len = min([len(frames) for frames in all_frames])
    for x in range(min_len):
        frame_each = [ all_frames[y][x] for y in range(len(all_frames)) ]
        new_frame  = np.concatenate(frame_each,axis=1)
        new_all_frames.append(new_frame)
    
    concat_clip = ImageSequenceClip(new_all_frames, fps=clip.fps)
    new_path = video_path.split('/')[-1]
    concat_clip.write_videofile(f'./merge/{new_path}')

    



In [ ]:
display_video_name = f'./merge/{new_path}'
display_video_mp4 = open('{}'.format(display_video_name),'rb').read()
display_video_data_url = "data:video/x-m4v;base64," + b64encode(display_video_mp4).decode()

print('Display result video: {}'.format(display_video_name), file=sys.stderr)
display(HTML("""
  <video width=800 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % display_video_data_url))